# Baseline model

Now that I have a good handle on what the linear regression features should look like, set up basic models.

The main goal here is to have a model structure that I can use for the remainder of this competition.

In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.core.pylabtools import figsize
 
import numpy as np
import pandas as pd
import polars as pl
import seaborn as sns

import statsmodels.formula.api as smf

import sklearn
sklearn.set_config(enable_metadata_routing=True)

In [2]:
__context__ = 'local'

import sys

if __context__ == 'local':
    trainDataLocation = '../../data/train/'
    libraryLocation = '../..'
    sys.path.append('..')
    from public_timeseries_testing_util import MockApi
    env = MockApi()
    

elif __context__ == 'kaggle':
    trainDataLocation = '/kaggle/input/predict-energy-behavior-of-prosumers/'
    libraryLocation = '/kaggle/input/'
    import enefit
    env = enefit.make_env()

sys.path.append(libraryLocation)

##  Pipeline writing

### Development principle:

From start to finish, train and test data should be sent through the same pipeline.
Train is allowed to 'fit'


In [3]:
train = pd.read_csv(trainDataLocation+'train.csv')
client = pd.read_csv(trainDataLocation+'client.csv')
weather_forecast = pd.read_csv(trainDataLocation + 'forecast_weather.csv')
prices_gas = pd.read_csv(trainDataLocation + 'gas_prices.csv')
prices_electricity = pd.read_csv(trainDataLocation + 'electricity_prices.csv')
solar = pd.read_csv(libraryLocation + '/enefittools/data/datasets/solar_data.csv')

### Data pipeline:

In [4]:
from enefittools.data import format_dfs

def assemble_train_client(train, client):
    return train.with_columns(
                    pl.col('prediction_datetime').dt.date().alias('date')
                ).join(client,
                       on=['county', 'product_type', 'is_business', 'date']
                ).with_columns(
                    (pl.col('target') / pl.col('installed_capacity')).alias('target_normalized')
                )


In [5]:
train, client, weather_forecast, prices_electricity, prices_gas, solar = \
        format_dfs(target=train, client=client, weather_forecast=weather_forecast,
                    gas_prices=prices_gas, electricity_prices=prices_electricity, solar=solar)

target_data = assemble_train_client(train, client)

### Features for regression

In [6]:
from sklearn.base import BaseEstimator, TransformerMixin
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess

class Datetime_Features(BaseEstimator, TransformerMixin):
    """Transformer for creating datetime features"""
    def __init__(self):
        pass
    
    def fit(self, target_data, y=None):
        dates = pd.date_range(target_data['prediction_datetime'].dt.date().min(), 
                      target_data['prediction_datetime'].dt.date().max())
        fourier = CalendarFourier(freq="A", order=6) 
        date_process = DeterministicProcess(
                                dates,
                                constant=False,
                                order=1,
                                additional_terms=[fourier],
                                drop=True)
        self.date_process_ = date_process

        _, date_names = self.make_date_features(dates)
        
        self.column_names_ = ['weekday', 'hour_of_day'] + date_names
        return self

    def make_date_features(self, target_dates):
        raw_features = self.date_process_.range(target_dates.min(), target_dates.max())
        raw_features = pl.from_pandas(raw_features.reset_index())
    
        name_dict = {old_name: f'{old_name[0:3]}_{old_name[4]}'
                     for old_name in raw_features.columns[-12:]}
    
        raw_features = raw_features.rename({'index': 'date'}
                                  ).rename(name_dict)

        return raw_features, ['trend'] + list(name_dict.values())

    
    def transform(self, full_data):
        date_features, _ = self.make_date_features(full_data['date'].unique())
        
        full_data = full_data.with_columns( 
                                pl.col('prediction_datetime').dt.weekday().alias('weekday'),
                                pl.col('prediction_datetime').dt.hour().alias('hour_of_day')
                            ).join(
                                    date_features.with_columns(pl.col('date').dt.date()),
                                    on='date'
                            )
        return full_data


In [7]:
import datetime

class Delayed_Features(BaseEstimator, TransformerMixin):
    """Transformer for adding time-shifted historical data"""
    def __init__(self, to_delay):
        self.to_delay = to_delay

    def fit(self, full_data, y=None):
        self.column_names_ = ['2d_ago', '7d_ago']
        return self

    def transform(self, full_data, historical=None):
        """Add time-shifted historical data"""
        provided_data = historical
        
        if historical is None:
            historical = full_data[[self.to_delay, 'prediction_unit_id', 'is_consumption', 'prediction_datetime']]
    
        out = full_data.with_columns( 
                            (pl.col('prediction_datetime') - datetime.timedelta(days=2)).alias('2d_ago'),
                            (pl.col('prediction_datetime') - datetime.timedelta(days=7)).alias('7d_ago')
                        ).join(historical,
                               left_on=['prediction_unit_id', 'is_consumption', '2d_ago'],
                               right_on=['prediction_unit_id', 'is_consumption', 'prediction_datetime'],
                               how='left', suffix='_2d_ago'
                        ).join(historical,
                               left_on=['prediction_unit_id', 'is_consumption', '7d_ago'],
                               right_on=['prediction_unit_id', 'is_consumption', 'prediction_datetime'],
                               how='left', suffix='_7d_ago'
                        ).drop('2d_ago', '7d_ago'
                        )
        if provided_data is None:
            return out.drop_nulls()
        else:
            return out.fill_null(0)



In [8]:
from sklearn.pipeline import Pipeline

regression_features_production = Pipeline([('time_features', Datetime_Features()),
                                ('ar_features', Delayed_Features('target_normalized').set_transform_request(historical=True) )])

regression_features_consumption = Pipeline([('time_features', Datetime_Features()),
                                ('ar_features', Delayed_Features('target').set_transform_request(historical=True) )])


In [9]:
# limit resources for local development
if __context__ == 'local':
    target_data = target_data.filter(pl.col('prediction_unit_id')== 0)


In [10]:
target_production = target_data.filter(pl.col('is_consumption') == False)
target_consumption = target_data.filter(pl.col('is_consumption') == True)

### Linear models

In [11]:
import statsmodels.formula.api as smf
from sklearn.base import BaseEstimator, RegressorMixin
import warnings

import sklearn
sklearn.set_config(enable_metadata_routing=True)




class SM_Regression(BaseEstimator, RegressorMixin):
    """ Wrapper for statsmodels formula OLS
    """
    def __init__(self, formula, to_drop):
        self.formula = formula
        self.to_drop = to_drop

        self.model = None
        self.is_fit = False

    def fit(self, data, overwrite=False):
        if self.is_fit and not overwrite:
            # protection against costly re-fitting
            raise Exception('Already fit')

        self.model = smf.ols(self.formula, data=data.to_pandas())
        self.model = self.model.fit()
        
        self.model.remove_data()
        self.is_fit = True
        self.is_fit_ = True
        return self
    
    def predict(self, X):
        predictions = self.model.predict(X.to_pandas())

        outputs = X.drop(
                        self.to_drop
                  ).with_columns(
                        prediction=pl.lit(pl.from_pandas(predictions))
                  )

        return outputs

    def residuals(self, data, target_col='target'):
        predictions = pl.from_pandas(self.predict(data))

        return data.with_columns(
                        prediction=pl.lit(predictions)
                  ).with_columns(
                        residual=pl.col(target_col)-pl.col('prediction')
                  )

In [12]:
features_production = regression_features_production.fit_transform(target_production)
features_consumption = regression_features_consumption.fit_transform(target_consumption)

In [13]:
# this is a feature of the formula approach that causes some troubles

M = Datetime_Features()
_=M.fit_transform( target_production)
date_cols = ' + '.join(M.column_names_[2:])

In [14]:
regression_cols = ['weekday', 'hour_of_day', 'target_2d_ago', 'target_7d_ago', 'trend'] + \
                  [f'sin_{i}' for i in range(1,7)] + [f'cos_{i}' for i in range(1,7)]
date_cols = " + ".join(regression_cols[4:])

consumption_spec =f'target ~ (C(weekday) + C(hour_of_day)) * ({date_cols}) + C(weekday)*target_2d_ago + target_7d_ago'
production_spec = f'target_normalized ~ C(hour_of_day) * ({date_cols}) + target_normalized_2d_ago'


consumption_model = SM_Regression(consumption_spec, regression_cols
                                 )
production_model = SM_Regression(production_spec, regression_cols
                                )

In [15]:
consumption_model.fit(features_consumption)

SM_Regression(formula='target ~ (C(weekday) + C(hour_of_day)) * (trend + sin_1 '
                      '+ sin_2 + sin_3 + sin_4 + sin_5 + sin_6 + cos_1 + cos_2 '
                      '+ cos_3 + cos_4 + cos_5 + cos_6) + '
                      'C(weekday)*target_2d_ago + target_7d_ago',
              to_drop=['weekday', 'hour_of_day', 'target_2d_ago',
                       'target_7d_ago', 'trend', 'sin_1', 'sin_2', 'sin_3',
                       'sin_4', 'sin_5', 'sin_6', 'cos_1', 'cos_2', 'cos_3',
                       'cos_4', 'cos_5', 'cos_6'])

In [16]:
production_model.fit(features_production)

SM_Regression(formula='target_normalized ~ C(hour_of_day) * (trend + sin_1 + '
                      'sin_2 + sin_3 + sin_4 + sin_5 + sin_6 + cos_1 + cos_2 + '
                      'cos_3 + cos_4 + cos_5 + cos_6) + '
                      'target_normalized_2d_ago',
              to_drop=['weekday', 'hour_of_day', 'target_2d_ago',
                       'target_7d_ago', 'trend', 'sin_1', 'sin_2', 'sin_3',
                       'sin_4', 'sin_5', 'sin_6', 'cos_1', 'cos_2', 'cos_3',
                       'cos_4', 'cos_5', 'cos_6'])

### Random Forest on the residuals

In [17]:
from sklearn.ensemble import RandomForestRegressor

#### What features do we want for our random forests?

Also, its worth considering ways to make the random forests (or other approaches) able to share parameter relevance information between locations.

In [18]:
def extract_rf_features(features):
    return features[['county', 'is_business', 'product_type', 'prediction_datetime',
                     'row_id', 'prediction_unit_id', 'date_when_predicting', 'eic_count',
                     'installed_capacity', 'residual']]





In [19]:
prices_gas = pd.read_csv(trainDataLocation + 'gas_prices.csv')

## Pipeline Assembly

Running the pipeline from scratch

In [3]:
# import and transform data
from enefittools.data.format_data import format_dfs, assemble_train_client

train = pd.read_csv(trainDataLocation+'train.csv')
client = pd.read_csv(trainDataLocation+'client.csv')
weather_forecast = pd.read_csv(trainDataLocation + 'forecast_weather.csv')
prices_gas = pd.read_csv(trainDataLocation + 'gas_prices.csv')
prices_electricity = pd.read_csv(trainDataLocation + 'electricity_prices.csv')
solar = pd.read_csv(libraryLocation + '/enefittools/data/datasets/solar_data.csv')

target_production, target_consumption, weather_forecast, prices_electricity, prices_gas, solar = \
        format_dfs(target=train, client=client, weather_forecast=weather_forecast,
                   gas_prices=prices_gas, electricity_prices=prices_electricity, solar=solar,
                   assemble_and_split=True
                  )


# limit resources for local development
if __context__ == 'local':
    target_production = target_production.filter(pl.col('prediction_unit_id')== 0)
    target_consumption = target_consumption.filter(pl.col('prediction_unit_id')== 0)


In [4]:
# feature pipeline
from enefittools.features.datetime_features import Datetime_Features
from enefittools.features.autoregressive_features import Delayed_Features


from sklearn.pipeline import Pipeline

regression_features_production = Pipeline([
                                    ('time_features', Datetime_Features()),
                                    ('ar_features', Delayed_Features('target'))
                                ])

regression_features_consumption = Pipeline([
                                    ('time_features', Datetime_Features()),
                                    ('ar_features', Delayed_Features('target'))
                                ])


In [5]:
# linear regression pipeline
from enefittools.models.linear_models import SM_Regression
from enefittools.features.target_transformers import Normalize_Target
from enefittools.models.chaining import Predictions_to_Features

regression_cols = ['weekday', 'hour_of_day', 'target_2d_ago', 'target_7d_ago', 'trend'] + \
                  [f'sin_{i}' for i in range(1,7)] + [f'cos_{i}' for i in range(1,7)]
date_cols = " + ".join(regression_cols[4:])

consumption_spec =f'target ~ (C(weekday) + C(hour_of_day)) * ({date_cols}) + C(weekday)*target_2d_ago + target_7d_ago'
production_spec = f'target ~ C(hour_of_day) * ({date_cols}) + target_2d_ago'


production_regression = Pipeline([('norm-fwd', Normalize_Target(mode='fwd')),
                                  ('extraction', regression_features_production),
                                  ('regression', Predictions_to_Features(
                                                      SM_Regression(production_spec, 
                                                                    to_drop=regression_cols)
                                  )),
                                  ('norm-inv', Normalize_Target(mode='inv'))
                                 ])
                                
consumption_regression = Pipeline([('extraction', regression_features_consumption),
                                 ('regression', SM_Regression(consumption_spec, to_drop=regression_cols))
                                  ])
                                

In [6]:
production_regression.fit(target_production)

Pipeline(steps=[('norm-fwd', Normalize_Target()),
                ('extraction',
                 Pipeline(steps=[('time_features', Datetime_Features()),
                                 ('ar_features',
                                  Delayed_Features(to_delay='target'))])),
                ('regression',
                 Predictions_to_Features(model=SM_Regression(formula='target ~ '
                                                                     'C(hour_of_day) '
                                                                     '* (trend '
                                                                     '+ sin_1 '
                                                                     '+ sin_2 '
                                                                     '+ sin_3 '
                                                                     '+ sin_4 '
                                                                     '+ sin_5 '
                                                                     '+ sin_6 '
                                                                     '+ cos_1 '
                                                                     '+ cos_2 '
                                                                     '+ cos_3 '
                                                                     '+ cos_4 '
                                                                     '+ cos_5 '
                                                                     '+ cos_6) '
                                                                     '+ '
                                                                     'target_2d_ago',
                                                             to_drop=['weekday',
                                                                      'hour_of_day',
                                                                      'target_2d_ago',
                                                                      'target_7d_ago',
                                                                      'trend',
                                                                      'sin_1',
                                                                      'sin_2',
                                                                      'sin_3',
                                                                      'sin_4',
                                                                      'sin_5',
                                                                      'sin_6',
                                                                      'cos_1',
                                                                      'cos_2',
                                                                      'cos_3',
                                                                      'cos_4',
                                                                      'cos_5',
                                                                      'cos_6']))),
                ('norm-inv', Normalize_Target(mode='inv'))])

In [7]:
consumption_regression.fit(target_consumption)

Pipeline(steps=[('extraction',
                 Pipeline(steps=[('time_features', Datetime_Features()),
                                 ('ar_features',
                                  Delayed_Features(to_delay='target'))])),
                ('regression',
                 SM_Regression(formula='target ~ (C(weekday) + C(hour_of_day)) '
                                       '* (trend + sin_1 + sin_2 + sin_3 + '
                                       'sin_4 + sin_5 + sin_6 + cos_1 + cos_2 '
                                       '+ cos_3 + cos_4 + cos_5 + cos_6) + '
                                       'C(weekday)*target_2d_ago + '
                                       'target_7d_ago',
                               to_drop=['weekday', 'hour_of_day',
                                        'target_2d_ago', 'target_7d_ago',
                                        'trend', 'sin_1', 'sin_2', 'sin_3',
                                        'sin_4', 'sin_5', 'sin_6', 'cos_1',
                                        'cos_2', 'cos_3', 'cos_4', 'cos_5',
                                        'cos_6']))])

## Run on the test set

In [8]:
from enefittools.data.format_predictions import format_outputs

iter_test = env.iter_test()

for (test, revealed_targets, client, weather_historical, weather_forecast,
    prices_electricity, prices_gas, sample_prediction) in iter_test:
    
    prod_test, consume_test, revealed_targets,weather_forecast, \
    prices_electricity, prices_gas, sample_prediction = \
    format_dfs(target=test, revealed_targets=revealed_targets, client=client,
               weather_forecast=weather_forecast, electricity_prices=prices_electricity,
               gas_prices=prices_gas, sample_prediction=sample_prediction,
              assemble_and_split=True, mode='test')

    prod_predictions = production_regression.predict(prod_test, historical=revealed_targets)
    consume_predictions = consumption_regression.predict(consume_test, historical=revealed_targets)

    prediction = format_outputs([prod_predictions, consume_predictions], sample_prediction)
    
    env.predict(prediction)